In [6]:
%pip install openai
%pip install --upgrade openai
%pip install langchain
%pip install PyPDF2
%pip install --upgrade --quiet  langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

In [7]:
import openai
from langchain.chat_models import ChatOpenAI
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import re
from PyPDF2 import PdfReader
import pandas as pd
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv
import pandas as pd


load_dotenv()

In [8]:

deployment_name=os.getenv("deployment_name")#'gpt-4-32k-rfmanrique'
version="2023-05-15"

api_key= os.getenv("api_key")
openai_api_key = api_key

api_version="2023-12-01-preview"
azure_endpoint="https://invuniandesai-2.openai.azure.com/"


llm = AzureChatOpenAI(deployment_name=deployment_name, openai_api_version=version, openai_api_key=api_key, openai_api_base=azure_endpoint)


In [ ]:
template = """
Tengo estas de oraciones en wayuu y español y quiero que revises si hace sentido que sean la misma oracion en distintos idiomas y devuelvas unicamente "True" si crees que son la misma oracion y "False" si crees que son distintas
Ejemplos

Input:
wayu: Makaa’ulain salii tepichikana.
español: Los niños no tienen chivos.
Output:
True

Input:
wayu: Kakaa’ulainshii tepichikana
español: No?
Output:
False

Caso:
wayu: {wy}
español: {es}
"""

llm = AzureChatOpenAI(deployment_name=deployment_name, openai_api_version=version, openai_api_key=api_key, openai_api_base=azure_endpoint)


/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/dep

In [9]:
def clean_data_set(x):
  try:
    msg = HumanMessage(content=template.format(wy=x['wy'],es=x['es']))
    gpt_response = llm(messages=[msg]).content
    print(f"wy:{x['wy']} es:{x['es']} asw:{gpt_response}")
  except:
    gpt_response = 'False'
  return gpt_response


In [10]:
csv_files =  ['72.  Vamos a hablar tomo II. WEB.csv',
  '8548776.csv',
  'pdfcoffee.com_la-conjugacion-del-verbo-en-la-lengua-wayuu-pdf-free.csv',
  'Compendio de la Gramática de la Lengua Wayuu ( PDFDrive ).csv'
  ]

dataframes = [pd.read_csv(file) for file in csv_files]
wayuu_esp_extraction = pd.concat(dataframes, ignore_index=True)

In [ ]:
clean_df = wayuu_esp_extraction.apply(clean_data_set, axis=1)

wy:Ee’irajüshi taya wanee jayeechi sümüin Marakariita. es:Yo canto un jayeechi para Margarita. asw:True
wy:Ne’rüinjachi taya tachukuwa ’aya. es:Ellos me van a ver de nuevo. asw:True
wy:Aya’lajeesü Tareesa wanee komputatoora tamüin. es:Teresa quiere comprar una computadora para mí. asw:True
wy:Waneepulu ma’in sukuwa’ipa tepijaaya: alojushi taya kaliina jee nalojuin taya na wayuukanairua – nümaka walitkai nümüin pürinsipechonkai. es:Mi vida es muy monótona: yo cazo gallinas y los hombres me cazan a mí – le dijo el zorro al principito. asw:True
wy:Washitshaanas hi ma’i wayuukai chii. es:Este hombre es muy rico. asw:True
wy:Paapaa susukoo tüü sümaa ja ’rai shikii nneerü. es:Toma esta mochila esta con cincuenta monedas. asw:True
wy:Japüishi chira jintüi kai, nnojoishi ekeein o ’upalaa. es:Ese niño tiene vergüenza, no quiere comer delante de alguien. asw:True
wy:Alu’u anaajaainja ’aya nneerü alijuna kai chira. es:Ese alijuna sí que guarda dinero. asw:True
wy:Teküinpu ’u majayüt kalü türa. es

In [12]:
wayuu_esp_extraction['is_good'] = clean_df.apply(lambda x: True if 'True' in x else False)

In [13]:
wayuu_esp_extraction.to_csv('dataset.csv', index=False)